# 1. Cài đặt thư viện


In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import plotly.express as px
import cv2
from PIL import Image

plt.style.use("seaborn")
plt.rcParams['figure.figsize'] = (13, 7)

<ipython-input-3-a314bd594e74>:11: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn")


# 2. Tải dữ liệu

In [ ]:
id_df = pd.read_csv("book_id.csv")
data_df = pd.read_csv("book_data.csv")
cmt_df = pd.read_csv("comments.csv")

Let focus on data_df (main file).

In [ ]:
data_df.head()

,product_id,title,authors,original_price,current_price,quantity,category,n_review,avg_rating,pages,manufacturer,cover_link
0,74021317,Cây Cam Ngọt Của Tôi,José Mauro de Vasconcelos,108000,64800,53075.0,Tiểu Thuyết,11481,5.0,244,Nhà Xuất Bản Hội Nhà Văn,https://salt.tikicdn.com/ts/product/5e/18/24/2...
1,74021317,Cây Cam Ngọt Của Tôi,José Mauro de Vasconcelos,108000,64800,53075.0,Tiểu Thuyết,11481,5.0,244,Nhà Xuất Bản Hội Nhà Văn,https://salt.tikicdn.com/ts/product/5e/18/24/2...
2,184466860,Hành Tinh Của Một Kẻ Nghĩ Nhiều,Nguyễn Đoàn Minh Thư,86000,59900,7929.0,Sách tư duy - Kỹ năng sống,780,4.8,184,Nhà Xuất Bản Thế Giới,https://salt.tikicdn.com/ts/product/d7/99/24/1...
3,73787185,Những Tù Nhân Của Địa Lý,Tim Marshall,210000,126000,17896.0,Lĩnh vực khác,3623,4.8,430,Nhà Xuất Bản Hội Nhà Văn,https://salt.tikicdn.com/ts/product/8d/96/9e/c...
4,52789367,Nhà Giả Kim (Tái Bản 2020),Paulo Coelho,79000,47400,24668.0,Tác phẩm kinh điển,5131,5.0,228,Nhà Xuất Bản Hà Nội,https://salt.tikicdn.com/ts/product/45/3b/fc/a...


In [ ]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1796 entries, 0 to 1795
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   product_id      1796 non-null   int64  
 1   title           1796 non-null   object 
 2   authors         1653 non-null   object 
 3   original_price  1796 non-null   int64  
 4   current_price   1796 non-null   int64  
 5   quantity        1751 non-null   float64
 6   category        1796 non-null   object 
 7   n_review        1796 non-null   int64  
 8   avg_rating      1796 non-null   float64
 9   pages           1546 non-null   object 
 10  manufacturer    1523 non-null   object 
 11  cover_link      1796 non-null   object 
dtypes: float64(2), int64(4), object(6)
memory usage: 168.5+ KB


In [ ]:
id_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2024 entries, 0 to 2023
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      2024 non-null   int64
dtypes: int64(1)
memory usage: 15.9 KB


Ta có 1796 cuốn sách không trùng lặp. Nhưng trong id_df, Ta lại có 2024 id. Lý do cho vấn đề này là trong bộ dữ liệu có một số vật phẩm không phải là sách, dù vậy thì số lượng của chúng khá ít và không đáng quan tâm.

# 3. Xử lý dữ liệu

## Xử lý dữ liệu khiếm khuyết hoặc bị thiếu

In [ ]:
  data_df.isnull().sum()

product_id          0
title               0
authors           143
original_price      0
current_price       0
quantity           45
category            0
n_review            0
avg_rating          0
pages             250
manufacturer      273
cover_link          0
dtype: int64

### Xóa các mục bị trùng lặp

In [ ]:
data_df = data_df.drop_duplicates(subset=['title'])

In [ ]:
data_df.authors.value_counts()

Nguyễn Nhật Ánh    24
Higashino Keigo    20
.                  18
Thích Nhất Hạnh    16
Haruki Murakami    15
                   ..
Urako Kanamori      1
Cổ Viên             1
Robert Winston      1
Yongchul Kwon       1
John C. Maxwell     1
Name: authors, Length: 1083, dtype: int64

Xóa các tác giả không đích danh, đại diện bằng dấu "."

In [ ]:
data_df.loc[data_df.authors == '.' , 'authors'] = "Unknown"

### Author

In [ ]:
data_df.authors.value_counts()

Nguyễn Nhật Ánh    24
Higashino Keigo    20
Unknown            18
Thích Nhất Hạnh    16
Haruki Murakami    15
                   ..
Urako Kanamori      1
Cổ Viên             1
Robert Winston      1
Yongchul Kwon       1
John C. Maxwell     1
Name: authors, Length: 1083, dtype: int64

In [ ]:
data_df.authors = data_df.authors.fillna("Unknown")

### Quantity

In [ ]:
data_df.quantity = data_df.quantity.fillna(np.mean(data_df.quantity))

### Category

In [ ]:
data_df.category.value_counts()

Sách tư duy - Kỹ năng sống                 292
Tiểu Thuyết                                133
Truyện ngắn - Tản văn - Tạp Văn            109
Sách nghệ thuật sống đẹp                    58
Sách kỹ năng làm việc                       55
                                          ... 
Shaman King - Tập 19                         1
Blue Period - Tập 07                         1
Komi - Nữ Thần Sợ Giao Tiếp - Tập 14         1
Bên Rặng Tuyết Sơn (Tái Bản)                 1
Kaguya-Sama: Cuộc Chiến Tỏ Tình - Tập 3      1
Name: category, Length: 354, dtype: int64

Ta thấy có rất nhiều giá trị bị sai. Nên ta chỉ xét 30 giá trị hàng đầu.

In [ ]:
keeping_values = list(pd.DataFrame(data_df.category.value_counts()[1:30]).T.columns)
keeping_values

['Tiểu Thuyết',
 'Truyện ngắn - Tản văn - Tạp Văn',
 'Sách nghệ thuật sống đẹp',
 'Sách kỹ năng làm việc',
 'Bài học kinh doanh',
 'Truyện trinh thám',
 'Sách Học Tiếng Anh',
 'Sách Marketing - Bán hàng ',
 'Sách tài chính, tiền tệ',
 'Tác phẩm kinh điển',
 'Truyện Giả tưởng - Huyền Bí - Phiêu Lưu',
 'Kiến thức - Bách khoa',
 'Sách Làm Cha Mẹ',
 'Lĩnh vực khác',
 'Truyện dài',
 'Kiến Thức Bách Khoa',
 'Văn học thiếu nhi',
 'Light novel',
 'Truyện kể cho bé',
 'Sách quản trị, lãnh đạo',
 'Truyện đam mỹ',
 'Tiểu sử - Hồi ký',
 'Lịch Sử Thế Giới',
 'Sách Chiêm Tinh - Horoscope',
 'Sách Học Tiếng Hoa',
 'Truyện tranh Ehon',
 'Sách kinh tế học',
 'Sách giáo dục',
 'Tô màu - Luyện chữ  ']

In [ ]:
def handle_category(category):
    if category not in keeping_values:
        return "Others"
    return category

data_df.category = data_df.category.apply(lambda category: handle_category(category))

### Pages

In [ ]:
def handle_pages(lst_pages):
    lst_int = []
    for page in lst_pages:
        try:
            lst_int.append(int(page))
        except:
            continue

    return np.mean(lst_int)

mean_pages = handle_pages(list(data_df.pages))

In [ ]:
data_df.pages = data_df.pages.fillna(str(mean_pages))

In [ ]:
# There is one product having the value of pages is "Cuốn".
data_df.query('pages == "Cuốn"')

,product_id,title,authors,original_price,current_price,quantity,category,n_review,avg_rating,pages,manufacturer,cover_link
1266,197639028,Động Từ Bất Quy Tắc Và Ngữ Pháp Tiếng Anh Căn Bản,Mai Lan Hương,18000,12000,136.0,Sách Học Tiếng Anh,2,5.0,Cuốn,Nhà Xuất Bản Đà Nẵng,https://salt.tikicdn.com/ts/product/28/8e/9e/0...


Đổi datatype

In [ ]:
data_df = data_df.query('pages != "Cuốn"')

In [ ]:
data_df.pages = data_df.pages.apply(lambda page: round(float(page)))
data_df.pages = data_df.pages.astype("int64")

<ipython-input-22-ea51b01eedfa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df.pages = data_df.pages.apply(lambda page: round(float(page)))
<ipython-input-22-ea51b01eedfa>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df.pages = data_df.pages.astype("int64")


In [ ]:
data_df.pages.describe()

count      1768.000000
mean        455.603507
std        4561.569635
min          10.000000
25%         220.000000
50%         332.000000
75%         456.000000
max      192000.000000
Name: pages, dtype: float64

In [ ]:
data_df[data_df.pages > 3000]

,product_id,title,authors,original_price,current_price,quantity,category,n_review,avg_rating,pages,manufacturer,cover_link
650,201687274,Có Một Nổi Buồn Vừa Ngang Qua Đây,Gã,88000,64900,114.0,Truyện ngắn - Tản văn - Tạp Văn,6,5.0,192000,Nhà Xuất Bản Văn Học,https://salt.tikicdn.com/ts/product/08/fd/a5/0...


In [ ]:
data_df = data_df.query('pages < 3000')

### Manufacturer

In [ ]:
data_df.manufacturer.value_counts()

Nhà Xuất Bản Thế Giới                            247
NXB Trẻ                                          143
Nhà Xuất Bản Hà Nội                              128
Nhà Xuất Bản Dân Trí                             126
Nhà Xuất Bản Kim Đồng                            113
Nhà Xuất Bản Hội Nhà Văn                         111
Nhà Xuất Bản Thanh Niên                           97
Nhà Xuất Bản Tổng hợp TP.HCM                      91
Nhà Xuất Bản Lao Động                             89
Nhà Xuất Bản Văn Học                              76
Nhà Xuất Bản Hồng Đức                             56
Nhà Xuất Bản Công Thương                          46
Nhà Xuất Bản Phụ Nữ Việt Nam                      38
Nhà Xuất Bản Phụ Nữ                               30
Nhà Xuất Bản Đà Nẵng                              16
Nhà Xuất Bản Đại Học Quốc Gia Hà Nội              13
Nhà Xuất Bản Mỹ Thuật                              8
Nhà Xuất Bản Tri Thức                              7
Nhà Xuất Bản Lao Động Xã Hội                  

Có vài nhà xuất bản giống nhau nhưng tên khá khác, ta tiến hành gộp chúng lại:
* Nhà Xuất Bản Hồng Đức & hong duc
* Nhà Xuất Bản Dân Trí & NXB Dân Trí
* Nhà Xuất Bản Đại Học Quốc Gia Hà Nội & ĐHQG Hà Nội


Let grouping them

In [ ]:
def handle_manufactuner(manufactuner):
    if manufactuner == "hong duc":
        return "Nhà Xuất Bản Hồng Đức"
    elif manufactuner == "NXB Dân Trí":
        return "Nhà Xuất Bản Dân Trí"
    elif manufactuner == "ĐHQG Hà Nội":
        return "Nhà Xuất Bản Đại Học Quốc Gia Hà Nội"
    else:
        return manufactuner

data_df.manufacturer = data_df.manufacturer.apply(lambda manufacturer: handle_manufactuner(manufacturer))

Xử lý những dữ liệu bị thiếu

In [ ]:
data_df.manufacturer = data_df.manufacturer.fillna("Unknown")

In [ ]:
data_df.head()

,product_id,title,authors,original_price,current_price,quantity,category,n_review,avg_rating,pages,manufacturer,cover_link
0,74021317,Cây Cam Ngọt Của Tôi,José Mauro de Vasconcelos,108000,64800,53075.0,Tiểu Thuyết,11481,5.0,244,Nhà Xuất Bản Hội Nhà Văn,https://salt.tikicdn.com/ts/product/5e/18/24/2...
2,184466860,Hành Tinh Của Một Kẻ Nghĩ Nhiều,Nguyễn Đoàn Minh Thư,86000,59900,7929.0,Others,780,4.8,184,Nhà Xuất Bản Thế Giới,https://salt.tikicdn.com/ts/product/d7/99/24/1...
3,73787185,Những Tù Nhân Của Địa Lý,Tim Marshall,210000,126000,17896.0,Lĩnh vực khác,3623,4.8,430,Nhà Xuất Bản Hội Nhà Văn,https://salt.tikicdn.com/ts/product/8d/96/9e/c...
4,52789367,Nhà Giả Kim (Tái Bản 2020),Paulo Coelho,79000,47400,24668.0,Tác phẩm kinh điển,5131,5.0,228,Nhà Xuất Bản Hà Nội,https://salt.tikicdn.com/ts/product/45/3b/fc/a...
5,147920903,Một Thoáng Ta Rực Rỡ Ở Nhân Gian,Unknown,135000,81000,10000.0,Tiểu Thuyết,1636,4.8,304,Nhà Xuất Bản Hội Nhà Văn,https://salt.tikicdn.com/ts/product/2f/b5/4e/a...


### Create new column(s)
Tạo thêm 1 cột tên discount để thể hiện % số tiền giảm giá

In [ ]:
data_df['discount'] = round(1 - data_df['current_price']/data_df['original_price'],2)

cân nhắc chuyển từ VND sang USD để con số phù hợp hơn cho việc tính toán

In [ ]:
def convert_to_usd(vietnamdong):
    usd = vietnamdong/23700
    usd = round(usd, 2)

    return usd

data_df.original_price = data_df.original_price.apply(lambda vietnamdong: convert_to_usd(vietnamdong))
data_df.current_price = data_df.current_price.apply(lambda vietnamdong: convert_to_usd(vietnamdong))

## Thay đổi datatype

In [ ]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1767 entries, 0 to 1795
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   product_id      1767 non-null   int64  
 1   title           1767 non-null   object 
 2   authors         1767 non-null   object 
 3   original_price  1767 non-null   float64
 4   current_price   1767 non-null   float64
 5   quantity        1767 non-null   float64
 6   category        1767 non-null   object 
 7   n_review        1767 non-null   int64  
 8   avg_rating      1767 non-null   float64
 9   pages           1767 non-null   int64  
 10  manufacturer    1767 non-null   object 
 11  cover_link      1767 non-null   object 
 12  discount        1766 non-null   float64
dtypes: float64(5), int64(3), object(5)
memory usage: 193.3+ KB


In [ ]:
data_df.title = data_df.title.astype("string")
data_df.authors = data_df.authors.astype("string")
data_df.category = data_df.category.astype("string")
data_df.manufacturer = data_df.manufacturer.astype("string")
data_df.cover_link = data_df.cover_link.astype("string")

data_df.quantity = data_df.quantity.apply(lambda quantity: round(float(quantity)))
data_df.quantity = data_df.quantity.astype("int64")

### Lưu lại data sau tiền xử lý

In [ ]:
data_df.to_csv("prepared_data_book.csv")